In [1]:
def extract_feature(file_name, mfcc, chroma, mel, spectral_rolloff, spectral_flux, zero_crossing_rate):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        if spectral_rolloff:
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T, axis=0)
            results = np.hstack((result, rolloff))
        if spectral_flux:
            flux = np.mean(librosa.onset.onset_strength(y=X, sr=sample_rate, hop_length=512, aggregate=np.median))
            results = np.hstack((result, flux))
        if zero_crossing_rate:
            zcr = np.mean(librosa.feature.zero_crossing_rate(y=X))
            results = np.hstack((result, zcr))
    return result

In [2]:
emotions={
  '01':'Neutral',
  '02':'Calm',
  '03':'Happy',
  '04':'Sad',
  '05':'Angry',
  '06':'Fear',
  '07':'Disgust',
  '08':'Surprised'
}
observed_emotions=['Fear','Disgust','Surprised']

In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
def load_data(test_size):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\HP\\SpeechData\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True, spectral_rolloff=True, spectral_flux=True, zero_crossing_rate=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
x_train,x_test,y_train,y_test=load_data(test_size=0.20)
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(460, 116)
Features extracted: 168


In [4]:
model=MLPClassifier(alpha=0.01, batch_size=100, epsilon=1e-08, hidden_layer_sizes=(100,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=100, learning_rate='adaptive',
              max_iter=500)

In [5]:
y_pred=model.predict(x_test)

In [6]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 79.31%


In [7]:
import joblib
joblib.dump(model, 'model.sav')

['model.sav']

In [8]:
import IPython.display as ipd
filename = 'model.sav'
loaded_model = joblib.load(filename)
fname = 'C:/Users/HP/Sample/Disgust.wav'
feature = extract_feature(fname, mfcc=True, chroma=True, mel=True, spectral_rolloff=True, spectral_flux=True, zero_crossing_rate=True)
feature = feature.reshape(1, -1)
emotion = loaded_model.predict(feature)[0]
print("Predicted emotion:", emotion)

Predicted emotion: Disgust


In [9]:
ipd.Audio(fname)

In [10]:
fname = 'C:/Users/HP/Sample/Fear.wav'
feature = extract_feature(fname, mfcc=True, chroma=True, mel=True, spectral_rolloff=True, spectral_flux=True, zero_crossing_rate=True)
feature = feature.reshape(1, -1)
emotion = loaded_model.predict(feature)[0]
print("Predicted emotion:", emotion)

Predicted emotion: Fear


In [11]:
ipd.Audio(fname)

In [12]:
fname = 'C:/Users/HP/Sample/Surprise.wav'
feature = extract_feature(fname, mfcc=True, chroma=True, mel=True, spectral_rolloff=True, spectral_flux=True, zero_crossing_rate=True)
feature = feature.reshape(1, -1)
emotion = loaded_model.predict(feature)[0]
print("Predicted emotion:", emotion)

Predicted emotion: Surprised


In [13]:
ipd.Audio(fname)

In [16]:
fname = 'C:/Users/HP/Sample/myaudio.wav'
feature = extract_feature(fname, mfcc=True, chroma=True, mel=True, spectral_rolloff=True, spectral_flux=True, zero_crossing_rate=True)
feature = feature.reshape(1, -1)
emotion = loaded_model.predict(feature)[0]
print("Predicted emotion:", emotion)

Predicted emotion: Fear


In [17]:
ipd.Audio(fname)